# El_hoy_18_04_23

#### 0 Import libraries

In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame
from pyspark.sql import functions as SqlFuncs
from pyspark.sql.functions import *
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_date


sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.38.1 
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::940173853583:role/AWSGluestudio-datalake
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: 86672604-cd57-4430-a394-5a908aaeb2e5
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.38.1
--enable-glue-datacatalog true
Waiting for session 86672604-cd57-4430-a394-5a908aaeb2e5 to get into ready status...
Session 86672604-cd57-4430-a394-5a908aaeb2e5 has been created.



#### 1 Import data

In [2]:
liq= (
    glueContext.create_dynamic_frame.from_catalog(
        database="gg-data-elhoy",
        table_name="liquidacion",
        transformation_ctx="liq",
    )
)

nota= (
    glueContext.create_dynamic_frame.from_catalog(
        database="gg-data-elhoy",
        table_name="nota",
        transformation_ctx="nota",
    )
)

dt= (
    glueContext.create_dynamic_frame.from_catalog(
        database="gg-data-elhoy",
        table_name="detalle_nota",
        transformation_ctx="dt",
    )
)

inv= (
    glueContext.create_dynamic_frame.from_catalog(
        database="gg-data-elhoy",
        table_name="inventario_gas",
        transformation_ctx="inv",
    )
)

carburacion= (
    glueContext.create_dynamic_frame.from_catalog(
        database="gg-data-elhoy",
        table_name="carburacion",
        transformation_ctx="globalnet_carburacion",
    )
)

In [3]:
dire = glueContext.create_dynamic_frame.from_options(
    format_options={"quoteChar": '"', "withHeader": True, "separator": ","},
    connection_type="s3",
    format="csv",
    connection_options={
        "paths": [
            "s3://globalgas-datalake/datasets/raw/raw/direccion/Direcciones.csv"
        ],
        "recurse": True,
    },
    transformation_ctx="dire",
)

In [4]:
mv= (
    glueContext.create_dynamic_frame.from_catalog(
        database="gg-data-elhoy",
        table_name="movimiento_gas_inventario",
        transformation_ctx="mv",
    )
)

In [5]:
gn3api = (
    glueContext.create_dynamic_frame.from_catalog(
        database="gg_datalake_db_prod",
        table_name="globalnet_gn3_api_3",
        transformation_ctx="gn3api",
    )
)

In [6]:
u = (
    glueContext.create_dynamic_frame.from_catalog(
        database="gg-data-elhoy",
        table_name="unidad",
        transformation_ctx="u",
    )
)

In [7]:
cm = (
    glueContext.create_dynamic_frame.from_catalog(
        database="gg-data-elhoy",
        table_name="concepto_movimiento_gas",
        transformation_ctx="cm",
    )
)

#### 0.1 Funcion para generar fechas

In [8]:
def generate_series(start, stop, interval):
    """
    :param start  - lower bound, inclusive
    :param stop   - upper bound, exclusive
    :interval int - increment interval in seconds
    """
    spark = SparkSession.builder.getOrCreate()
    # Determine start and stops in epoch seconds
    start, stop = spark.createDataFrame(
        [(start, stop)], ("start", "stop")
    ).select(
        [col(c).cast("timestamp").cast("long") for c in ("start", "stop")
    ]).first()
    # Create range with increments and cast to timestamp
    return spark.range(start, stop, interval).select(
        col("id").cast("timestamp").alias("value")
    )

#### 2 Convert to dataframe

In [22]:
inv0 = inv.toDF().select("planta", "id_planta","fecha_hora","diferencia","traspaso_entrada", "traspaso_salida", "gas_empleado", "inventario_inicial", "inventario_final", "compras", "ventas","existencia_fisica","eliminado_logico", "venta_planta")
#.distinct()
inv00 = inv0.filter(inv0.eliminado_logico == "NO")
#Proceso consepto moviniento gas
cm0 = cm.toDF().drop('fecha_extraccion')
cm1 = cm0
cm2 = cm1.select("*").distinct()

#Cambiar a dataframe
liq00 = liq.toDF()
n00 = nota.toDF()
liq0 = liq00.filter(liq00.eliminado_logico == "NO")
n0 = n00.filter(n00.eliminado_logico == "NO") 
d0 = dt.toDF()
u0 = u.toDF()

In [23]:
dire1 = dire.toDF().select('Direccion','Entidad_Legal', 'planta_datalake')

In [24]:
liq1 = liq0.withColumn('planta_datalake', concat(lit('Planta'),liq0.planta))

### 3 Movimiento gas inventario

In [25]:
# Proceso movimiento gas
mv0 = mv.toDF()
mv1 = mv0.filter(mv0.eliminado_logico == 'NO')
mv2 = mv1.withColumnRenamed("id_concepto_movimiento_gas","id_concepto_movimiento_gas1")
#.withColumnRenamed("planta_datalake", "planta_datalake1")
mv2 = mv2.withColumn('planta_datalake1', concat( lit('Planta'),mv2.planta)).withColumnRenamed("id_planta", "id_planta1")

In [26]:
#join
cm3 = cm2.filter(cm2.tipo_inventario == "OTRAS_SALIDAS").select('*').distinct()
cm3 = cm3.withColumn('planta_datalake', concat(lit('Planta'),cm3.planta))

In [27]:

cmga = cm3.join(mv2, (cm3.id_concepto_movimiento_gas==mv2.id_concepto_movimiento_gas1) & (cm3.id_planta == mv2.id_planta1), "inner")
#Agrupacion

In [28]:
cmga = cmga.select('id_liquidacion',coalesce(cmga['planta_datalake'],cmga['planta_datalake1']),'kilos', 'tipo_inventario').withColumnRenamed( 'coalesce(planta_datalake, planta_datalake1)', 'planta_datalake')
#.withColumnRenamed( 'kilos', 'kilos_donativo')

In [29]:
cmg00 = cmga.fillna(0)
cmg01 = cmg00.groupBy("id_liquidacion", "planta_datalake").sum("kilos").withColumnRenamed("sum(kilos)","kilos_donativo")

cmg02 = cmg01

In [30]:
#rename id_liquidacion, id_nota, id_unidad
n1 = n0.withColumnRenamed("id_liquidacion", "id_liquidacion1").withColumnRenamed("id_planta", "id_planta1").drop('planta')
#.withColumn('planta_datalake1',concat(lit('Planta'), n0.planta) ).withColumnRenamed("id_planta", "id_planta1")
#.withColumnRenamed("planta_datalake", "planta_datalake1")

d1 = d0.withColumnRenamed("id_nota", "id_nota1").withColumnRenamed("id_planta", "id_planta1").drop('planta')
#.withColumn("planta_datalake1", concat(lit('Planta'), d0.planta))
#.withColumnRenamed("planta_datalake", "planta_datalake1")

u1 = u0.withColumnRenamed("id_unidad","id_unidad1").withColumnRenamed("id_planta", "id_planta1").drop('planta')
#.withColumnRenamed("planta_datalake","planta_datalake1")

cmg1 = cmg02.withColumnRenamed("id_liquidacion", "id_liquidacion1").withColumnRenamed("id_planta", "id_planta1").drop('planta')
#.withColumnRenamed("planta_datalake", "planta_datalake1").withColumnRenamed("id_planta", "id_planta1")

### 3.01 ver como vienen los donativos

#### 3.1 VALE

In [38]:
cm3 = cm2.filter(cm2.tipo_inventario == "GAS_EMPLEADO")
cm3 = cm3.withColumn('planta_datalake', concat(lit('Planta'),cm3.planta)).drop('empresa', 'planta')
cmga2 = cm3.join(mv2, (cm3.id_concepto_movimiento_gas==mv2.id_concepto_movimiento_gas1) & (cm3.planta_datalake == mv2.planta_datalake1), "inner")
#Agrupacion
cmga3 = cmga2.fillna(0)
#cmga2.fillna(0)
cmga3 = cmga3.select(coalesce(cmga3["planta_datalake"],cmga3["planta_datalake1"]), 'id_concepto_movimiento_gas', 'tipo_inventario', 'nombre', 'id_planta','empresa', 'planta', 'eliminado_logico', 'id_concepto_movimiento_gas1', 'id_liquidacion', 'kilos', 'fecha_movimiento', 'diferencia', 'id_planta1', 'fecha_extraccion').withColumnRenamed("coalesce(planta_datalake, planta_datalake1)", "planta_datalake")

In [39]:
#cmga3.filter(cmga3.planta_datalake == 'PlantaGuzman').filter(cmga3.fecha_movimiento.between('2023-08-22', '2023-08-22 23:59:59')).select('eliminado_logico', 'kilos', 'id_concepto_movimiento_gas').show()

+----------------+-----+--------------------------+
|eliminado_logico|kilos|id_concepto_movimiento_gas|
+----------------+-----+--------------------------+
|              NO|150.0|                        15|
+----------------+-----+--------------------------+


In [6]:
#cmga3.select(coalesce(cmga3["planta_datalake"],cmga3["planta_datalake1"]), 'id_concepto_movimiento_gas', 'tipo_inventario', 'nombre', 'id_planta','empresa', 'planta', 'eliminado_logico', 'id_concepto_movimiento_gas1', 'id_liquidacion', 'kilos', 'fecha_movimiento', 'diferencia', 'id_planta1', 'fecha_extraccion').withColumnRenamed("coalesce(planta_datalake, planta_datalake1)", "planta_datalake1")

In [19]:
#cmga4= (cmga3.withColumn("fecha_hora_mx",  from_utc_timestamp(cmga3.fecha_movimiento,"GMT-5")))
#when(inv00.fecha_hora <='2023-04-08',).otherwise(from_utc_timestamp(inv00.fecha_hora, ))))  "America/Mexico_City"
cmga5 = cmga3.withColumn("fecha_movimiento_day", to_date(col("fecha_movimiento")))
cmga6 = cmga5.groupBy( "planta_datalake", 'fecha_movimiento_day').sum("kilos").withColumnRenamed("sum(kilos)","vale").withColumnRenamed("planta_datalake","planta_datalake1")

In [7]:
#cmga6.filter(cmga6.planta_datalake1 == 'PlantaPlantaFlogas').select('fecha_movimiento_day',  'planta_datalake1','vale').show()

In [20]:
cmga5.columns

['planta_datalake', 'id_concepto_movimiento_gas', 'tipo_inventario', 'nombre', 'id_planta', 'empresa', 'planta', 'eliminado_logico', 'id_concepto_movimiento_gas1', 'id_liquidacion', 'kilos', 'fecha_movimiento', 'diferencia', 'id_planta1', 'fecha_extraccion', 'fecha_movimiento_day']


In [21]:
cmga5

DataFrame[planta_datalake: string, id_concepto_movimiento_gas: bigint, tipo_inventario: string, nombre: string, id_planta: bigint, empresa: string, planta: string, eliminado_logico: string, id_concepto_movimiento_gas1: bigint, id_liquidacion: bigint, kilos: double, fecha_movimiento: timestamp, diferencia: bigint, id_planta1: bigint, fecha_extraccion: timestamp, fecha_movimiento_day: date]


#### 4 Liq-nota-detalle_nota- unidad-movimiento gas

In [22]:
#join liq-nota
ln = liq1.join(n1, (liq1.id_liquidacion == n1.id_liquidacion1) & (liq1.id_planta == n1.id_planta1), "fullouter" )
ln1 = ln.select("planta",'id_planta', "id_liquidacion", "fecha_liquidacion","id_nota", "id_unidad")


##join liq-nota-detallenota
ld = ln1.join(d1, (ln1.id_nota == d1.id_nota1) & (ln1.id_planta == d1.id_planta1), "fullouter" )
ld1 = ld.select("planta",'id_planta', "id_liquidacion", "fecha_liquidacion","id_nota", "id_unidad", "kilos")

##Join liq-nota-detallenota-unidad
lu = ld1.join(u1, (ld1.id_unidad == u1.id_unidad1) & (ld1.id_planta == u1.id_planta1), "fullouter")
lu1 = lu.select("planta",'id_planta' ,"id_liquidacion", "fecha_liquidacion","id_nota", "id_unidad", "kilos", "tipo_unidad")
### agrupacion lu
lu2 = lu1.fillna(0)
lu3 = lu1.groupBy( "planta", "id_liquidacion", "fecha_liquidacion", "id_unidad","tipo_unidad").sum("kilos").withColumnRenamed("sum(kilos)", "kilos")
lu3 = lu3.withColumn('planta_datalake',concat(lit('Planta'), lu3.planta) )

In [8]:
#lu3.filter(lu3.planta =='PlantaFlogas').show()

In [23]:
# liq.. nota... kilos_donativo
cmg03 = cmg02.withColumnRenamed("id_liquidacion","id_liquidacion1").withColumnRenamed("planta_datalake","planta_datalake1")
lc2 = lu3.join(cmg03, (lu3.id_liquidacion == cmg03.id_liquidacion1) & (lu3.planta_datalake == cmg03.planta_datalake1), "fullouter")
lc3 = lc2.select("planta_datalake", "id_liquidacion",'id_liquidacion1', "fecha_liquidacion", "id_unidad", "kilos", "tipo_unidad", "kilos_donativo")

In [9]:
#cmg03.columns

In [10]:
#lc3.filter(lc3.planta_datalake =='PlantaPlantaFlogas').show()

In [11]:
#lc3.filter(lc3.planta_datalake == 'PlantaTepic').filter(lc3.fecha_liquidacion >= '2023-01-03').filter(lc3.fecha_liquidacion <'2023-01-04').filter(lc3.kilos_donativo.isNotNull()).orderBy('id_liquidacion').show()
#.filter(lc3.fecha_liquidacion >= '2023-01-01').filter(lc3.fecha_liquidacion <'2023-01-02')

##### 4.1 Cambio de fecha diaria... liq-nota-detallenota-unidad-mov_gas

In [24]:
lc33 = lc3.withColumn("fecha_liquidacion_day",to_date(col("fecha_liquidacion")))
lc4 = lc33.select("*")
lc5 = lc4.drop("fecha_liquidacion")
lc6 = lc5.drop("fecha_liquidacion_mx")

lc7 = lc6.groupBy("id_unidad", "tipo_unidad", "planta_datalake", "fecha_liquidacion_day" ).sum("kilos_donativo", "kilos")
lc8 = lc7.withColumnRenamed("sum(kilos)","kilos").withColumnRenamed("sum(kilos_donativo)","kilos_donativo")

In [12]:
#lc8.columns

In [13]:
#lc8.filter(lc8.planta_datalake =='PlantaPlantaFlogas').show()

In [25]:
car1 = carburacion.toDF()

In [26]:
car2 = car1.withColumn("planta_datalake", concat(lit('Planta'), car1.planta))

#### 5 Carburación

In [27]:
#carburacion.toDF()
car2 = car2.select("fecha_carburacion", "carburacion", "planta_datalake", "id_unidad", "eliminado_logico")
car2 = car2.filter(car2.eliminado_logico == 'NO')
#car3 = (car2.withColumn("fecha_car_mx",  from_utc_timestamp(car2.fecha_carburacion, "GMT-5")))
#America/Mexico_City
#when(car2.fecha_carburacion <='2023-04-08',).otherwise(from_utc_timestamp(car2.fecha_carburacion, "GMT-5"))))

In [28]:
car4 = car2.withColumn("fecha_car_day",to_date(col("fecha_carburacion")))
car5 = (car4.withColumn("cons_propio", round(car4.carburacion * .54, 2)))
car6 = car5.select("cons_propio", "planta_datalake", "id_unidad", "fecha_car_day")
car7 = car6.groupBy("planta_datalake", "id_unidad", "fecha_car_day").sum("cons_propio")
car8 = car7.withColumnRenamed("sum(cons_propio)","cons_propio")
car9 = car8.withColumnRenamed("planta_datalake", "planta_datalake1").withColumnRenamed("id_unidad","id_unidad1")

### 5.01 guardar para ver el desgloce por mes

##### 5.1 liq-nota--carburacion

In [29]:
lca = lc8.join(car9, (lc8.planta_datalake == car9.planta_datalake1) & (lc8.id_unidad == car9.id_unidad1) & (lc8.fecha_liquidacion_day == car9.fecha_car_day), "fullouter")
lca1 = lca.select(coalesce(lca["planta_datalake"],lca["planta_datalake1"]), "fecha_car_day","fecha_liquidacion_day",coalesce(lca["id_unidad"],lca["id_unidad1"]), "kilos", "kilos_donativo", "cons_propio", "tipo_unidad" )
lca2 = lca1.withColumnRenamed("coalesce(planta_datalake, planta_datalake1)", "planta_datalake1").withColumnRenamed("fecha_liquidacion_day","fecha_liquidacion").withColumnRenamed("coalesce(id_unidad, id_unidad1)", "id_unidad")

## 

In [30]:
#lca2.filter(lca2.planta_datalake1 =='PlantaPlantaFlogas').show
inv00.columns

['planta', 'id_planta', 'fecha_hora', 'diferencia', 'traspaso_entrada', 'traspaso_salida', 'gas_empleado', 'inventario_inicial', 'inventario_final', 'compras', 'ventas', 'existencia_fisica', 'eliminado_logico', 'venta_planta']


### AQUI

#### 6 Inventario

In [37]:
inv1= (inv00.withColumn("planta_datalake",  concat(lit('Planta'), inv00.planta)))
#when(inv00.fecha_hora <='2023-04-08',).otherwise(from_utc_timestamp(inv00.fecha_hora, ))))  "America/Mexico_City"
inv2 = inv1.withColumn("fecha_day", to_date(col("fecha_hora")))
inv3 = inv2.groupBy("planta_datalake", "fecha_day").sum("traspaso_entrada", "traspaso_salida", "gas_empleado", "inventario_inicial", "inventario_final", "compras", "existencia_fisica", "ventas", "venta_planta", "diferencia")
inv4 = inv3.withColumnRenamed("sum(traspaso_entrada)", "traspaso_entrada").withColumnRenamed("sum(traspaso_salida)", "traspaso_salida").withColumnRenamed("sum(gas_empleado)", "gas_empleado").withColumnRenamed("sum(inventario_inicial)", "inventario_inicial").withColumnRenamed("sum(inventario_final)", "inventario_final").withColumnRenamed("sum(compras)", "compras").withColumnRenamed("sum(existencia_fisica)", "existencia_fisica").withColumnRenamed("fecha_day", "fecha_liquidacion1").withColumnRenamed("sum(ventas)", "ventas").withColumnRenamed("sum(venta_planta)", "venta_planta").withColumnRenamed("sum(diferencia)", "diferencia")

In [38]:
inv4.filter(inv4.planta_datalake == 'PlantaCardenas').filter(inv4.fecha_liquidacion1.between('2023-07-01', '2023-07-31')).groupBy('planta_datalake').sum('diferencia').show()

+---------------+------------------+
|planta_datalake|   sum(diferencia)|
+---------------+------------------+
| PlantaCardenas|-468.6299999999999|
+---------------+------------------+


In [14]:
#lca2.filter(lca2.planta_datalake1 =='PlantaPlantaFlogas').show()

##### 6.1 Liq-nota-...inventario

Este join es simplemente para el caso cuando no se pudo hacer la migracion o no hay datos en la tabla de nota, se hace el uso de la columna ventas en la tabla inventario.
No tiene caso pivotear todas las cantidades que provienen de la tabla de compras debido a que el costo de la memoria es elevado (tardado y costoso a nivel recursos de la nube)

In [32]:
lca3 = lca2.join(inv4, (lca2.planta_datalake1 == inv4.planta_datalake) & (lca2.fecha_liquidacion== inv4.fecha_liquidacion1), "fullouter")
lca11 = lca3.select(coalesce(lca3["planta_datalake"],lca3["planta_datalake1"]), coalesce(lca3["fecha_liquidacion"],lca3["fecha_liquidacion1"]), 'id_unidad', 'kilos', 'kilos_donativo', 'cons_propio', 'tipo_unidad', 'traspaso_entrada', 'inventario_inicial', 'inventario_final', 'compras', 'existencia_fisica', 'ventas', 'venta_planta', "diferencia" )
lca22 = lca11.withColumnRenamed("coalesce(planta_datalake, planta_datalake1)", "planta_datalake").withColumnRenamed("coalesce(fecha_liquidacion, fecha_liquidacion1)","fecha_liquidacion")
lca23 = lca22.withColumn('kilos2', when(lca2.kilos ==0, lca22.ventas).otherwise(lca22.kilos)).drop('kilos').withColumnRenamed('kilos2', 'kilos')

In [57]:
#lca23.filter(lca23.planta_datalake =='PlantaPlantaFlogas').show()
#lca2.columns

['planta_datalake', 'fecha_liquidacion', 'id_unidad', 'kilos_donativo', 'cons_propio', 'tipo_unidad', 'traspaso_entrada', 'inventario_inicial', 'inventario_final', 'compras', 'existencia_fisica', 'ventas', 'venta_planta', 'diferencia', 'kilos']


#### 8  kilos donativo

In [34]:
kilos_d = lca23.select("tipo_unidad",  "planta_datalake", "fecha_liquidacion", "kilos_donativo")
kilos_dp = kilos_d.groupBy("planta_datalake", "fecha_liquidacion").sum("kilos_donativo")
kilos_dp = kilos_dp.fillna(0)
kilos_dp2 = kilos_dp.withColumnRenamed('sum(kilos_donativo)', 'kilos_donativo') 
#kilos_dp1 = kilos_dp.withColumn("kilos_donativo", kilos_dp.null+ kilos_dp.ANDEN + kilos_dp.ESTACION +kilos_dp.ESTACIONARIO + kilos_dp.MAYORISTA_E + kilos_dp.MAYORISTA_P +kilos_dp.PORTATIL + kilos_dp.UTILITARIO)
#kilos_dp2 = kilos_dp1.select( "planta_datalake", "fecha_liquidacion", "kilos_donativo")

#### 9 consumo propio

In [35]:
kilos_cp1 = car8.drop('id_unidad').withColumnRenamed('fecha_car_day', 'fecha_liquidacion')
kilos_cp2 = kilos_cp1.groupBy("planta_datalake", "fecha_liquidacion").sum("cons_propio").withColumnRenamed('sum(cons_propio)', 'cons_propio')

#### 7 Pivot kilos, tipo de venta

In [36]:
kilos = lca23.select("tipo_unidad",  "planta_datalake", "fecha_liquidacion", "kilos")
kilos = kilos.fillna(0)
kilosp = kilos.groupBy("planta_datalake", "fecha_liquidacion").pivot("tipo_unidad").sum("kilos")
kilosp = kilosp.fillna(0)
kilosp1 = kilosp.withColumn("kilos", kilosp.null+ kilosp.ANDEN +kilosp.ESTACION+kilosp.ESTACIONARIO + kilosp.MAYORISTA_E + kilosp.MAYORISTA_P +kilosp.PORTATIL + kilosp.UTILITARIO)

In [15]:
#kilosp1.filter(kilosp1.planta_datalake =='PlantaPlantaFlogas').show() 

#### 10 Union de los pivots

In [37]:
#kilosp1, kilos_dp2, kilos_cp2
kilos_dp3 =  kilos_dp2.withColumnRenamed("planta_datalake","planta_datalake1").withColumnRenamed("fecha_liquidacion","fecha_liquidacion1")
k1 = kilosp1.join(kilos_dp3, (kilosp1.planta_datalake ==  kilos_dp3.planta_datalake1) & (kilosp1.fecha_liquidacion ==  kilos_dp3.fecha_liquidacion1), "inner")
k2 = k1.drop("planta_datalake1").drop("fecha_liquidacion1")
kilos_cp3 =  kilos_cp2.withColumnRenamed("planta_datalake","planta_datalake1").withColumnRenamed("fecha_liquidacion","fecha_liquidacion1")
k3 = k2.join(kilos_cp3, (k2.planta_datalake ==  kilos_cp3.planta_datalake1) & (k2.fecha_liquidacion ==  kilos_cp3.fecha_liquidacion1), "inner")
k4 = k3.drop("planta_datalake1").drop("fecha_liquidacion1")

#### 11 Retiros

In [38]:
gn30 = gn3api.toDF().select("date", "planta_datalake", "totalquantity", "unitType")
gn31 =  gn30.withColumn("fecha_day", to_date(col("date")))
gn32 = gn31.groupBy("fecha_day", "planta_datalake").pivot("unitType").sum("totalquantity").withColumnRenamed("sum(totalquantity)", "totalquantity").withColumnRenamed("fecha_day", "fecha_liquidacion2").withColumnRenamed("planta_datalake","planta_datalake1")
gn32 = gn32.withColumn('planta_datalake2', when(gn32.planta_datalake1 == 'PlantaFlogas', 'PlantaPlantaFlogas').otherwise(gn32.planta_datalake1)).drop('planta_datalake1').withColumnRenamed('planta_datalake2', 'planta_datalake1')

In [16]:
#gn32.filter(gn32.planta_datalake1 =='PlantaFlogas').show()
#gn32.show()

In [39]:
#gn32.filter(gn32.planta_datalake1 =='PlantaPlantaFlogas').show()
gn32 = gn32.withColumnRenamed("PORTATIL", "PORTATIL_e").withColumnRenamed("ESTACION", "ESTACION_e").withColumnRenamed("ESTACIONARIO", "ESTACIONARIO_e").withColumnRenamed("MAYORISTA_E", "MAYORISTA_E_e").withColumnRenamed("ANDEN", "ANDEN_e").withColumnRenamed("MAYORISTA_P", "MAYORISTA_P_e").fillna(0)
gn32 = gn32.withColumn('totalquantity',  gn32.ANDEN_e +gn32.ESTACION_e+gn32.ESTACIONARIO_e + gn32.MAYORISTA_E_e + gn32.MAYORISTA_P_e+ gn32.PORTATIL_e)

In [17]:
#gn31.filter(gn31.planta_datalake == 'PlantaTepic').filter(gn31.fecha_day == '2023-04-14').groupBy('fecha_day').sum('totalquantity').show()

In [18]:
#gn32.filter(gn32.planta_datalake1 == 'PlantaTepic').filter(gn32.fecha_liquidacion2 == '2023-04-14').show()
#gn32.columns

##### 11.1 Retiros  e inventario

In [58]:
gi = inv4.join(gn32, (inv4.planta_datalake == gn32.planta_datalake1 ) & (inv4.fecha_liquidacion1 == gn32.fecha_liquidacion2), "fullouter")
gi2 = gi.select(coalesce(gi['planta_datalake'], gi['planta_datalake1']), coalesce(gi['fecha_liquidacion1'],gi['fecha_liquidacion2']), 'traspaso_entrada', 'traspaso_salida', 'gas_empleado', 'inventario_inicial', 'inventario_final', 'compras','diferencia', 'existencia_fisica','venta_planta','ANDEN_e', 'ESTACION_e', 'ESTACIONARIO_e', 'MAYORISTA_E_e', 'MAYORISTA_P_e', 'PORTATIL_e', 'totalquantity')
gi3 = gi2.withColumnRenamed("coalesce(planta_datalake, planta_datalake1)", "planta_datalake1").withColumnRenamed("coalesce(fecha_liquidacion1, fecha_liquidacion2)", "fecha_liquidacion1")

In [19]:
#gi3.filter(gi3.planta_datalake1 =='PlantaPlantaFlogas').show()

Aqui ver cuales son las columnas

In [59]:
ki = k4.join(gi3, (k4.planta_datalake == gi3.planta_datalake1) & (k4.fecha_liquidacion == gi3.fecha_liquidacion1), "fullouter")
ki1 = ki.select(coalesce(ki['planta_datalake'], ki['planta_datalake1']), coalesce(ki['fecha_liquidacion1'],ki['fecha_liquidacion']),'diferencia',"null","ANDEN","ESTACION","ESTACIONARIO","MAYORISTA_E","MAYORISTA_P","PORTATIL","UTILITARIO",'ANDEN_e', 'ESTACION_e', 'ESTACIONARIO_e', 'MAYORISTA_E_e', 'MAYORISTA_P_e', 'PORTATIL_e',"kilos","kilos_donativo", 'traspaso_entrada', 'traspaso_salida', 'gas_empleado', 'inventario_inicial', 'inventario_final', 'compras', 'existencia_fisica', 'venta_planta','totalquantity', "cons_propio")
ki2 = ki1.withColumnRenamed("coalesce(planta_datalake, planta_datalake1)","planta_datalake").withColumnRenamed("coalesce(fecha_liquidacion1, fecha_liquidacion)","fecha_liquidacion")
#ki3 = fromDF(ki2, glue_ctx, ki4)

In [24]:
#ki2.filter(ki2.planta_datalake =='PlantaPlantaFlogas').show()

#### 12 Generar la serie de tiempo para tener todos los registros

In [42]:
df = generate_series("2022-08-11", "2023-04-17", 60 * 60 * 24)
df1 = df.withColumn("current_date",current_date())
l1 = df1.select("current_date").collect()
df2 = generate_series("2017-01-01", l1[0].current_date, 60 * 60 * 24)
df3 = df2.withColumn("fecha", to_date(col("value")))
df4 = df3.drop("value")
liq3 = liq.toDF()

In [43]:
liq3 = liq3.withColumn('planta_datalake', concat(lit('Planta'), liq3.planta))

In [44]:
a = liq3.select("planta_datalake").distinct().orderBy("planta_datalake").collect()

In [45]:
liq3.columns

['empresa', 'planta', 'fecha_liquidacion', 'id_liquidacion', 'id_unidad', 'eliminado_logico', 'id_planta', 'fecha_extraccion', 'planta_datalake']


In [46]:
df5 = df4.withColumn("planta_datalake", lit(a[0].planta_datalake))
for i in range(len(a)):
    df6 = df4.withColumn("planta_datalake", lit(a[i].planta_datalake))
    df5 = df5.union(df6)

df7 =  df5.distinct()
df8 = df7.withColumnRenamed("planta_datalake","planta_datalake1")

#### 13 Union con todos los dias

In [66]:
ki4 = ki2.join(df8, (ki2.fecha_liquidacion == df8.fecha) & (ki2.planta_datalake == df8.planta_datalake1), "fullouter")
ki5 = ki4.select(coalesce(ki4['planta_datalake'], ki4['planta_datalake1']), coalesce(ki4['fecha'], ki4['fecha_liquidacion']), 'diferencia','null', 'ANDEN', 'ESTACION', 'ESTACIONARIO', 'MAYORISTA_E', 'MAYORISTA_P', 'PORTATIL', 'UTILITARIO', 'kilos', 'kilos_donativo', 'traspaso_entrada', 'traspaso_salida', 'gas_empleado', 'inventario_inicial', 'inventario_final', 'compras', 'existencia_fisica','venta_planta', 'totalquantity','ANDEN_e', 'ESTACION_e', 'ESTACIONARIO_e', 'MAYORISTA_E_e', 'MAYORISTA_P_e', 'PORTATIL_e', 'cons_propio')
ki6 = ki5.withColumnRenamed("coalesce(planta_datalake, planta_datalake1)", "planta_datalake").withColumnRenamed("coalesce(fecha, fecha_liquidacion)", "fecha_liquidacion")

#vt

In [67]:
cmga6 = cmga6.withColumnRenamed('planta_datalake', 'planta_datalake1')

In [68]:
cmga6 = cmga6.filter(cmga6.fecha_movimiento_day >='2022-01-01')

In [28]:
#cmga6.filter(cmga6.planta_datalake1 == 'PlantaPlantaFlogas').show()

### 12.1 Join con el vale

In [69]:
ki7 = ki6.join(cmga6, ((ki6.planta_datalake == cmga6.planta_datalake1) & (ki6.fecha_liquidacion == cmga6.fecha_movimiento_day)), 'outer')

In [29]:
#ki7.filter(ki7.planta_datalake == 'PlantaPlantaFlogas').select('planta_datalake', 'planta_datalake1').show()

In [70]:
ki8 = ki7.select(coalesce(ki7['planta_datalake'], ki7['planta_datalake1']), coalesce(ki7['fecha_movimiento_day'],ki7['fecha_liquidacion']),'diferencia','null', 'ANDEN', 'ESTACION', 'ESTACIONARIO', 'MAYORISTA_E', 'MAYORISTA_P', 'PORTATIL', 'UTILITARIO', 'kilos', 'kilos_donativo', 'traspaso_entrada', 'traspaso_salida', 'gas_empleado', 'inventario_inicial', 'inventario_final', 'compras', 'existencia_fisica', 'venta_planta','ANDEN_e', 'ESTACION_e', 'ESTACIONARIO_e', 'MAYORISTA_E_e', 'MAYORISTA_P_e', 'PORTATIL_e', 'totalquantity', 'cons_propio',  'vale')
ki9 = ki8.withColumnRenamed('coalesce(planta_datalake, planta_datalake1)', 'planta_datalake').withColumnRenamed('coalesce(fecha_movimiento_day, fecha_liquidacion)', 'fecha_liquidacion')
k7 = ki9
#.fillna(0)
ki8 =  k7.withColumn('planta_datalake2', when(k7.planta_datalake == 'PlantaPlantaFlogas', 'PlantaFlogas').otherwise(k7.planta_datalake)).drop('planta_datalake').withColumnRenamed('planta_datalake2', 'planta_datalake')

In [71]:
ki8.columns

['fecha_liquidacion', 'diferencia', 'null', 'ANDEN', 'ESTACION', 'ESTACIONARIO', 'MAYORISTA_E', 'MAYORISTA_P', 'PORTATIL', 'UTILITARIO', 'kilos', 'kilos_donativo', 'traspaso_entrada', 'traspaso_salida', 'gas_empleado', 'inventario_inicial', 'inventario_final', 'compras', 'existencia_fisica', 'venta_planta', 'ANDEN_e', 'ESTACION_e', 'ESTACIONARIO_e', 'MAYORISTA_E_e', 'MAYORISTA_P_e', 'PORTATIL_e', 'totalquantity', 'cons_propio', 'vale', 'planta_datalake']


In [27]:
#ki8.filter(ki8['planta_datalake'] == 'PlantaFlogas').show()

In [53]:
ki7.columns

['planta_datalake', 'fecha_liquidacion', 'null', 'ANDEN', 'ESTACION', 'ESTACIONARIO', 'MAYORISTA_E', 'MAYORISTA_P', 'PORTATIL', 'UTILITARIO', 'kilos', 'kilos_donativo', 'traspaso_entrada', 'traspaso_salida', 'gas_empleado', 'inventario_inicial', 'inventario_final', 'compras', 'existencia_fisica', 'venta_planta', 'totalquantity', 'ANDEN_e', 'ESTACION_e', 'ESTACIONARIO_e', 'MAYORISTA_E_e', 'MAYORISTA_P_e', 'PORTATIL_e', 'cons_propio', 'planta_datalake1', 'fecha_movimiento_day', 'vale']


### Agregar zona, y entidad

In [72]:
dire2 = dire1.withColumnRenamed('planta_datalake','planta_datalake1')
k8 = ki8.join(dire2, (ki8.planta_datalake == dire2.planta_datalake1), 'outer').drop('planta_datalake1')

In [28]:
#k8.filter(k8['planta_datalake'] == 'PlantaFlogas').show()

In [73]:
k8.repartition('planta_datalake').write.mode("overwrite").format("parquet").save("s3://globalgas-datalake/cleaning_stepts/operaciones3/")